## ImageSets


In [1]:
import os
import random
 
trainval_percent = 0.1
train_percent = 0.9
xmlfilepath = 'data/VOC2007/Annotations'  
txtsavepath = 'data/VOC2007/ImageSets\Main'  
total_xml = os.listdir(xmlfilepath)
 
num = len(total_xml)
list = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)
 
if not os.path.exists(txtsavepath):
    print('not exist...{}'.format(txtsavepath))
    os.makedirs(txtsavepath)
    
ftrainval = open('data/VOC2007/ImageSets/Main/trainval.txt', 'w')  
ftest = open('data/VOC2007/ImageSets/Main/test.txt', 'w')  
ftrain = open('data/VOC2007/ImageSets/Main/train.txt', 'w')  
fval = open('data/VOC2007/ImageSets/Main/val.txt', 'w') 
 
for i in list:
    name = total_xml[i][:-4] + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftest.write(name)
        else:
            fval.write(name)
    else:
        ftrain.write(name)
        
ftrainval.close()
ftrain.close()
fval.close()
ftest.close()


## python voc_annotation.py

In [2]:
import xml.etree.ElementTree as ET
from os import getcwd

sets=[('2007', 'train'), ('2007', 'val'), ('2007', 'test')]

classes = ["方標"]     #修改为自己的类别

def convert_annotation(year, image_id, list_file):
    in_file = open('data/VOC%s/Annotations/%s.xml'%(year, image_id),encoding="utf-8")
    tree=ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

wd = getcwd()

for year, image_set in sets:
    image_ids = open('data/VOC%s/ImageSets/Main/%s.txt'%(year, image_set)).read().strip().split()
    list_file = open('%s_%s.txt'%(year, image_set), 'w',encoding="utf-8")
    for image_id in image_ids:
        list_file.write('%s/data/VOC%s/JPEGImages/%s.jpg'%(wd, year, image_id))
        convert_annotation(year, image_id, list_file)
        list_file.write('\n')
    list_file.close()



In [3]:
list_file

<_io.TextIOWrapper name='2007_test.txt' mode='w' encoding='utf-8'>

## 4.3 修改類別文件
找到model_data/voc_classes.txt文件，將裡邊的類別修改為自己的類別

## 4.4 修改anchor文件

我們知道yolo是利用9個anchor來檢測目標的位置，因此這9個anchor設置的好壞，會直接影響到訓練與檢測效果。

我們使用kmeans方法來聚類得到與我們數據集接近的anchors，在keras-yolov3下邊有一個kmeans.py文件，打開後將filename改為2007_train.txt（2處），保存運行該腳本，會得到9個anchors的坐標值以及準確率。記下這9個坐標值，然後打開model_data中的yolo_anchors.txt，按照文本中的格式，將9個anchors值依次寫入即可


In [4]:
import numpy as np
 
class YOLO_Kmeans:
 
    def __init__(self, cluster_number, filename):
        self.cluster_number = cluster_number
        self.filename = "data/2007_train.txt"
 
    def iou(self, boxes, clusters):  # 1 box -> k clusters
        n = boxes.shape[0]
        k = self.cluster_number
 
        box_area = boxes[:, 0] * boxes[:, 1]
        box_area = box_area.repeat(k)
        box_area = np.reshape(box_area, (n, k))
 
        cluster_area = clusters[:, 0] * clusters[:, 1]
        cluster_area = np.tile(cluster_area, [1, n])
        cluster_area = np.reshape(cluster_area, (n, k))
 
        box_w_matrix = np.reshape(boxes[:, 0].repeat(k), (n, k))
        cluster_w_matrix = np.reshape(np.tile(clusters[:, 0], (1, n)), (n, k))
        min_w_matrix = np.minimum(cluster_w_matrix, box_w_matrix)
 
        box_h_matrix = np.reshape(boxes[:, 1].repeat(k), (n, k))
        cluster_h_matrix = np.reshape(np.tile(clusters[:, 1], (1, n)), (n, k))
        min_h_matrix = np.minimum(cluster_h_matrix, box_h_matrix)
        inter_area = np.multiply(min_w_matrix, min_h_matrix)
 
        result = inter_area / (box_area + cluster_area - inter_area)
        return result
 
    def avg_iou(self, boxes, clusters):
        accuracy = np.mean([np.max(self.iou(boxes, clusters), axis=1)])
        return accuracy
 
    def kmeans(self, boxes, k, dist=np.median):
        box_number = boxes.shape[0]
        distances = np.empty((box_number, k))
        last_nearest = np.zeros((box_number,))
        np.random.seed()
        clusters = boxes[np.random.choice(
            box_number, k, replace=False)]  # init k clusters
        while True:
 
            distances = 1 - self.iou(boxes, clusters)
 
            current_nearest = np.argmin(distances, axis=1)
            if (last_nearest == current_nearest).all():
                break  # clusters won't change
            for cluster in range(k):
                clusters[cluster] = dist(  # update clusters
                    boxes[current_nearest == cluster], axis=0)
 
            last_nearest = current_nearest
 
        return clusters
 
    def result2txt(self, data):
        f = open("yolo_anchors.txt", 'w')
        row = np.shape(data)[0]
        for i in range(row):
            if i == 0:
                x_y = "%d,%d" % (data[i][0], data[i][1])
            else:
                x_y = ", %d,%d" % (data[i][0], data[i][1])
            f.write(x_y)
        f.close()
 
    def txt2boxes(self):
        f = open(self.filename, 'r')
        dataSet = []
        for line in f:
            infos = line.split(" ")
            length = len(infos)
            for i in range(1, length):
                width = int(infos[i].split(",")[2]) - \
                    int(infos[i].split(",")[0])
                height = int(infos[i].split(",")[3]) - \
                    int(infos[i].split(",")[1])
                dataSet.append([width, height])
        result = np.array(dataSet)
        f.close()
        return result
 
    def txt2clusters(self):
        all_boxes = self.txt2boxes()
        result = self.kmeans(all_boxes, k=self.cluster_number)
        result = result[np.lexsort(result.T[0, None])]
        self.result2txt(result)
        print("K anchors:\n {}".format(result))
        print("Accuracy: {:.2f}%".format(
            self.avg_iou(all_boxes, result) * 100))
 
 
if __name__ == "__main__":
    cluster_number = 9
    filename = "data/2007_train.txt"  # This is the file that records all box sizes.
    kmeans = YOLO_Kmeans(cluster_number, filename)
    kmeans.txt2clusters()


K anchors:
 [[ 11  11]
 [ 16  15]
 [ 24  24]
 [ 33  36]
 [ 44  45]
 [ 59  60]
 [ 91  91]
 [147 144]
 [267 253]]
Accuracy: 81.01%
